In [ ]:
# Defining random seeds to enable reproducibility
from numpy.random import seed
seed(1)

import tensorflow as tf
tf.random.set_seed(1)
 
import random
random.seed(1)

import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
import tensorflow_addons as tfa
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.python.client import device_lib
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split


print(device_lib.list_local_devices())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


def load_dataset():
    """
    Loads the datasets encoded in .pkl files and returns its decoded form.

    Returns
    -------
    list
        A list of n-dimensional arrays representing the subjects samples that will be used to \\
        train the drunkenness classification model.
    ndarray
        A n-dimensional array representing the samples labels.
    """

    print("Loading Sober-Drunk Face Dataset, from Patras University")
    
    # Defining the sample and label sets filenames
    sets = [
        "Insert the x_imbalanced.pkl file path here",
        "Insert the y_imbalanced.pkl file path here"   
    ]
    
    # Defining an empty list for storing the decoded dataset
    loaded_datasets = []
 
    # Iterating over the dataset files
    for set_ in sets:
        # Opening the .pkl file in read mode
        with open(set_, 'rb') as file:
            # Appending the decoded dataset to the dataset list
            loaded_datasets.append(pickle.load(file))
    
    # Unpacking the dataset list into individual subsets
    x, y = loaded_datasets
    
    # Converting the label list to the n-dimensional array format
    y_arr= np.array(y)
    
    # Printing the dataset length for sanity check
    print("\nSamples total: {0}".format(len(x)))   
    
    # Slicing the frame sequences of each subject for selecting the
    # frames sampled at each 5 Hz
    x = x[::5]
    y_arr = y_arr[::5]
    
    # Printing the dataset length after slicing for sanity check
    print("\nSamples total after slicing: {0}".format(len(x)))
    
    # Returning the samples set and its respective labels
    return x, y_arr


def min_max_norm(dataset):
    """
    Normalizes the keyframes according to the minimum-maximum norm, \\
    such that pixel values ranges from 0 to 1.

    Parameters
    ----------
    dataset : list
        A list of n-dimensional arrays representing the subjects keyframes.

    Returns
    -------
    ndarray
        A n-dimensional array representing keyframes with pixel values ranging from 0 to 1.
    """

    # Converting the dataset type from list to n-dimensional array
    dataset = np.asarray(dataset, dtype="int16")

    # Finding the keyframes minimum and maximum values
    x_min = dataset.min(axis=(1, 2), keepdims=True)
    x_max = dataset.max(axis=(1, 2), keepdims=True)

    # Applying the minimum-maximum norm to each keyframe
    norm_dataset = (dataset - x_min) / (x_max - x_min)

    # Printing the minimum and maximum values from a given sample for sanity check
    print("\nMinMax normalization")
    print("dataset shape: ", norm_dataset.shape)
    print("min: ", norm_dataset[0].min())
    print("max: ", norm_dataset[0].max())

    # Returning the normalized dataset
    return norm_dataset


def visualize_model_history(hist):
    """
    Displays the training process loss and accuracy history.

    Parameters
    ----------
    hist : dictionary
        A dictionary conataining loss and accuracy values lists from the training and \\
        test sets.
    """

    # Displaying and saving the training and test loss history
    plt.figure()
    plt.plot(hist.history['loss'], linewidth=2, alpha=0.85)
    plt.plot(hist.history['val_loss'], linewidth=2, alpha=0.85)
    sns.despine()
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper right', frameon=False)
    plt.savefig("loss-curve.pdf", dpi=600, bbox_inches='tight', pad_inches=0.1)
    
    # Displaying and saving the training and test accuracy history
    plt.figure()
    plt.plot(hist.history['accuracy'], linewidth=2, alpha=0.85)
    plt.plot(hist.history['val_accuracy'], linewidth=2, alpha=0.85)
    sns.despine()
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='lower right', frameon=False)
    plt.savefig("accuracy-curve.pdf", dpi=600, bbox_inches='tight', pad_inches=0.1)
    
    # Showing the training process loss and accuracy history
    plt.show()


def feature_extraction(x_train):
    """
    Uses a pre-trained model to extract common features from facial thermal images. \\
    Such features will be used in the transfer learning step to train a binary \\
    classification model, which is expected to abstract new knowledge from these \\ 
    generic representations.

    Parameters
    ----------
    x_train : ndarray
        A n-dimensional array representing the training samples.
    
    Returns
    -------
    ndarray
        A n-dimensional array representing the flattened training feature maps.
    ndarray
        A n-dimensional array describing the flattened feature maps shape.
    """

    print("\nLoading pre-trained model")

    # Loading the base drunkenness classification model
    model = models.load_model('sober-drunk-cc_vgg16_ft-model_data_aug_tts_70-30.h5')
    
    # Printing the model summary
    model.summary()
    
    print("\nRemoving top layers")

    # Removing the base drunkenness classifier output layer
    model = keras.Sequential(model.layers[:-2])
    
    # Printing the base model structure again for sanity check
    model.summary()

    # Defining the base model output as the last pooling layer feature maps
    output = model.layers[-1].output
    
    # Adding a Flatten layer to the output
    output = Flatten()(output)
    
    # Defining a functional model for feature extraction
    model = models.Model(model.input, output)

    # Changing the model 'trainable' parameter to False
    model.trainable = False

    print("\n")

    # Iterating over the model layers
    for layer in model.layers:
        # Freezing the layers weights
        layer.trainable = False
        # Printing the layers 'trainable' parameter for sanity check
        print("{}: {}".format(layer.name, layer.trainable))

    # Extracting the training features
    train_features = model.predict(x_train, verbose=1)

    print('\nTrain Bottleneck Features: ', train_features.shape)

    # Defining the flattened fetaure maps shape used by the build_model_cv function
    features_shape = train_features[0].shape 
    
    # Returning the training features and its array shape
    return train_features, features_shape


def build_model_cv(features_shape):
    """
    Builds a keras sequential model and returns it.

    Parameters
    ----------
    features_shape : ndarray
        A n-dimensional array describing the flattened feature maps shape.

    Returns
    -------
    Sequential
        A multi-layer binary classification model.
    """
    
    # Defining a Sequential model
    model = models.Sequential()
    
    # Adding an input layer to receive the flattened feature array
    model.add(InputLayer(input_shape=features_shape, name="input"))
    
    # Adding a hidden layer to abstract non-linear features
    model.add(Dense(units=512, activation='relu', activity_regularizer=keras.regularizers.L2(0.001), name="hidden_layer_1"))

    # Defining an output layer to classify the received features as sober (0) or drunk (1)
    model.add(Dense(1, activation='sigmoid', name="output"))

    # Defining the optmization function
    adam = optimizers.Adam(learning_rate=1e-7, amsgrad=False)

    # Defining F-Measure as the classification performance metric
    f_measure = tfa.metrics.F1Score(num_classes=1, threshold=0.5, average=None, name='f_measure')
    
    # Compiling the model
    model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=False),
                  optimizer=adam,
                  metrics=f_measure)
    
    # Printing the model summary
    model.summary()
    
    # Returning the sequential model
    return model
    

def hyperparameter_search():
    """
    Defines a custom training loop to enable the k-fold cross-validation \\
    during the hidden units hyperparameter search.

    Notes
    -----
    Since we need to search the value of only one hyperparameter (hidden units) \\
    and the search space is small ([2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]), \\
    we performed the hyperparameter search manually in order to be able to analyze the \\
    model behavior more carefully when changing the number of hidden units.
    """
    
    # Loading the Sober-Drunk Dataset samples
    x, y = load_dataset()
    
    # Defining the training and test subsets using the random train-test split strategy
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1, stratify=y)

    # Applying the min-max normalization
    x_train = min_max_norm(x_train)
    x_test = min_max_norm(x_test)

    # Reshaping datsets to the tensor format (channel last)
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 3)
    x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 3)

    # Extracting features from the training subset
    train_features, features_shape = feature_extraction(x_train)
    
    print("\n")
    
    # Defining the stratified cross-validation folds
    folds = list(StratifiedKFold(n_splits=5, shuffle=False, random_state=None).split(train_features, y_train))
    
    # Defining the early stopping callback
    callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, min_delta=0, restore_best_weights=True, verbose=1)
 
    # Instantiating an empty list to store the model classification performance on each fold
    # this list will be used to calculate the model average performance throughout the
    # stratified cross-validation process
    fold_acc = []
    
    # Iterating over the stratified cross-validation folds
    for j, (train_idx, val_idx) in enumerate(folds):
        print('\nFold ',j)
        
        # building a keras sequential model with a hyperparameter manually defined
        model = model_build_cv(features_shape)

        # Defining the training and validation sets
        X_train_cv, y_train_cv = train_features[train_idx], y_train[train_idx]
        X_valid_cv, y_valid_cv = train_features[val_idx], y_train[val_idx]
        
        # Defining weights for sober and drunk classes in order to prevent the class  
        # imbalance from influencing the loss function
        zeros = np.count_nonzero(y_train_cv == 0)
        ones = np.count_nonzero(y_train_cv == 1)
    
        training_set_size = len(X_train_cv)
    
        weight_for_0 = (1 / zeros) * (training_set_size) / 2.0 
        weight_for_1 = (1 / ones) * (training_set_size) / 2.0
    
        class_weights = {0: weight_for_0, 1: weight_for_1}
        
        print("\nTraining with {0} samples and validating with {1} samples\n".format(len(X_train_cv), len(X_valid_cv)))

        # Fitting the model
        history = model.fit(x=X_train_cv, y=y_train_cv, 
                            validation_data=(X_valid_cv, y_valid_cv),
                            shuffle=False,
                            batch_size=120,  
                            epochs=400,
                            class_weight=class_weights,
                            callbacks=[callback],
                            verbose=1)
        
        # Evaluating the model on validation data
        val_loss, val_acc = model.evaluate(X_valid_cv, y_valid_cv, verbose=0)
        
        # Appending the model classification performance on the k-th fold to the cross-validation
        # accuracies list
        fold_acc.append(val_acc)
 
        # printing the model loss and classification performance on the k-th fold
        print('\nFold ',j)
        print("\nValidation F-Measure: ", val_acc)
        print("Validation loss: ", val_loss)

        print("\nClassification report: ")

        # Obtaining the validation samples class probabilities
        y_prob = model.predict(X_valid_cv)
        # Obtaining the binary label from the model output probabilities
        y_hat = (y_prob > 0.5).astype(int)

        # Printing the classification performance report
        report = classification_report(X_valid_cv, y_valid_cv, target_names=['sober', 'drunk'])
        print(report)
    
        print("\nConfusion Matrix: ")

        # Printing the confusion matrix
        matrix = confusion_matrix(y_valid_cv, y_hat)
        print(matrix)
    
        tn, fp, fn, tp = matrix.ravel()
        print("\nTrue Negatives: ", tn)
        print("False Positives: ", fp)
        print("False Negatives: ", fn)
        print("True Positives: ", tp)
        
        # Displaying the training process history
        visualize_model_history(history)

    # Printing the model classification F-Measure on each fold of the stratified 
    # cross-validation
    print("\nK-Fold F-Measures: ", fold_acc)
    
    # Printing the model average F-Measure along the stratified k-fold cross-validation 
    # procedure
    print("\nAverage F-Measure: ", np.mean(fold_acc))
    
    # Printing the standard deviation of F-Measures obtained throughout the 
    # stratified k-fold cross-validation procedure
    print("K-Fold Standard deviation: ", np.std(fold_acc))


def build_model():
    """
    Loads the base drunkenness classification model, \\
    freezes the convolutional layers weights, removes \\
    the output layer, adds a new classifier on top of \\
    convolutional layers, re-compiles the model and returns it.

    Returns
    -------
    Sequential
        A multi-layer binary classification model.
    """

    print("\nLoading pre-trained model")
    
    # Loading the base drunkenness classification model
    model = models.load_model('sober-drunk-cc_vgg16_ft-model_data_aug_tts_70-30.h5')
    
    # Printing the model summary
    model.summary()
    
    print("\nRemoving top layers and freezing weights\n")

    # Removing the base drunkenness classifier output layer
    model = keras.Sequential(model.layers[:-2])
    
    # Iterating over the model layers
    for layer in model.layers:
        # Freezing the layers weights
        layer.trainable = False
        # Printing the layers 'trainable' parameter for sanity check
        print("{}: {}".format(layer.name, layer.trainable))
 
    print("\nAdding new dense layers\n")

    # Adding a Flatten layer to the base model output
    model.add(Flatten(name="flatten"))
    
    # Adding a hidden layer to the base model output
    model.add(Dense(units=512, activation='relu', activity_regularizer=keras.regularizers.L2(0.001), name="hidden_layer_1"))
    
    # Adding an output layer to classify the received features as sober (0) or drunk (1)
    model.add(Dense(units=1, activation='sigmoid', name="output"))
    
    # Iterating over the model layers
    for layer in model.layers:
        # Printing the layers 'trainable' parameter for sanity check
        print("{}: {}".format(layer.name, layer.trainable))
    
    print("\n")
    
    # Defining the optmization function
    adam = optimizers.Adam(learning_rate=1e-7, amsgrad=False)
    
    # Defining F-Measure as the classification performance metric
    f_measure = tfa.metrics.F1Score(num_classes=1, threshold=0.5, average=None, name='f_measure')
    
    # Compiling the model
    model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=False),
                  optimizer=adam,
                  metrics=f_measure)
    
    # Printing the model summary
    model.summary()
    
    # Returning the sequential model
    return model

    
def transfer_learning():
    """
    Performs the transfer learning strategy from a previously trained base \\ 
    drunkenness classification model. \\ 
    
    In order to leverage the base model representation capability, we transferred \\ 
    this basic knowledge to a more robust classifier capable to identify \\ 
    drunkenness-related features variations. For such, we considered images from all \\
    acquisitions periods, in which the pixels behavior tends to change over time \\ 
    accordingly to the subjects BAC.
    """    
    
    # Loading the Sober-Drunk Dataset samples
    x, y = load_dataset()
    
    # Defining the training and test subsets using the random train-test split strategy
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1, stratify=y)
    
    # Applying the min-max normalization
    x_train = min_max_norm(x_train)
    x_test = min_max_norm(x_test)
    
    # Reshaping datsets to the tensor format (channel last)
    x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 3)
    x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], x_test.shape[2], 3)
    
    # Defining a functional model for transfer learning
    model = build_model()
    
    # Defining weights for sober and drunk classes in order to prevent the class  
    # imbalance from influencing the loss function
    zeros = np.count_nonzero(y_train == 0)
    ones = np.count_nonzero(y_train == 1)
    
    training_set_size = len(x_train)
    
    weight_for_0 = (1 / zeros) * (training_set_size) / 2.0 
    weight_for_1 = (1 / ones) * (training_set_size) / 2.0
    
    class_weights = {0: weight_for_0, 1: weight_for_1}
    
    # Fitting the model
    history = model.fit(x=x_train, y=y_train,
                        validation_data=(x_test, y_test),
                        shuffle=False,
                        batch_size=120, 
                        epochs=400,
                        class_weight=class_weights,
                        verbose=1)
    
    # Evaluating the model on unseen data
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=1)
    
    # Saving the obtained final drunkenness classification model
    model.save('sober-drunk-cc_vgg16_final-model-tl_512-hu_f-measure_tts_70-30_L2.h5')
    
    print("\nClassification report: ")
    
    # Obtaining the test samples class probabilities
    y_prob = model.predict(x_test)
    # Obtaining the binary label from the model output probabilities
    y_hat = (y_prob > 0.5).astype(int)
    
    # Printing the classification performance report
    report = classification_report(y_test, y_hat, target_names=['sober', 'drunk'])
    print(report)
    
    print("\nConfusion Matrix: ")

    # Printing the confusion matrix
    matrix = confusion_matrix(y_test, y_hat)
    print(matrix)
    
    tn, fp, fn, tp = matrix.ravel()
    print("\nTrue Negatives: ", tn)
    print("False Positives: ", fp)
    print("False Negatives: ", fn)
    print("True Positives: ", tp)
    
    # Displaying the training process history
    visualize_model_history(history)


# Running the transfer learning
transfer_learning()